The dataset used to make the dashboard contains the following columns:

 - Date: The date of the observation.
 - Recession: A binary variable indicating recession perion; 1 means it was recession, 0 means it was normal.
 - Automobile_Sales: The number of vehicles sold during the period.
 - GDP: The per capita GDP value in USD.
 - unemployment_rate: The monthly unemployment rate.
 - Consumer_Confidence: A synthetic index representing consumer confidence, which can impact consumer spending and automobile purchases.
 - Seasonality_Weight: The weight representing the seasonality effect on
 automobile sales during the period.
 - Price: The average vehicle price during the period.
 - Advertising_Expenditure: The advertising expenditure of the company.
 - Vehicle_Type: The type of vehicles sold; Supperminicar, Smallfamiliycar, Mediumfamilycar, Executivecar, Sports.
 - Competition: The measure of competition in the market, such as the number of competitors or market share of major manufacturers.
 - Month: Month of the observation extracted from Date.
 - Year: Year of the observation extracted from Date.


## Setting Up Libraries

In [1]:
!pip install pandas dash
!pip3 install httpx==0.20 dash plotly

import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 5.6 MB/s eta 0:00:00


## Make the Dashboard

In [2]:
#Load the data
data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')

#Initialize the Dash app
app = dash.Dash(__name__)

#Create the layout of the app
app.layout = html.Div([
    html.H1('Automobile Sales Statistics Dashboard', style={'textAlign': 'center', 'color': '#503D36', 'font-size': 24}),
    html.Div([
        html.Label('Select Statistics:'),
        #Report type dropdown
        dcc.Dropdown(
            id='dropdown-statistics',
            options=[
                {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
                {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
                ],
            placeholder='Select a report type'
        )
    ]),
    #Year dropdown
    html.Div(dcc.Dropdown(
            id='select-year',
            options=[{'label': i, 'value': i} for i in range(1980, 2024, 1)],
            placeholder='Select year'
        )),
    html.Div(id='output-container', className='chart-grid')
])

#Callback for disabling year dropdown when 'Recession Period Statistics' is chosen
@app.callback(
    Output(component_id='select-year', component_property='disabled'),
    Input(component_id='dropdown-statistics',component_property='value'))

def update_input_container(selected_statistics):
    if selected_statistics == 'Yearly Statistics':
        return False
    else:
        return True

#Callback for plotting
@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='select-year', component_property='value'), Input(component_id='dropdown-statistics', component_property='value')])


def update_output_container(input_year, selected_statistics):
    if selected_statistics == 'Recession Period Statistics':
        #Filter the data for recession periods
        recession_data = data[data['Recession'] == 1]

        #Create and display graphs for recession period statistics
        yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        R_chart1 = dcc.Graph(figure=px.line(yearly_rec, x='Year', y='Automobile_Sales', title="Average Automobile Sales Fluctuation over Recession Period"))

        average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        R_chart2  = dcc.Graph(figure=px.bar(average_sales, x='Vehicle_Type', y='Automobile_Sales', title='Average Number of Vehicles Sold By Vehicle Type During Recession'))

        exp_rec = recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        R_chart3 = dcc.Graph(figure=px.pie(exp_rec, values='Advertising_Expenditure', names='Vehicle_Type', title='Total Expenditure Share By Vehicle Type During Recessions'))

        R_chart4 = dcc.Graph(figure=px.bar(recession_data, x='unemployment_rate', y='Automobile_Sales', color='Vehicle_Type', title='Effect of Unemployment Rate on Vehicle Type and Sales'))

        return [
            html.Div(className='chart-item', children=[html.Div(children=R_chart1),html.Div(children=R_chart2)], style={'display': 'flex'}),
            html.Div(className='chart-item', children=[html.Div(children=R_chart3),html.Div(children=R_chart4)], style={'display': 'flex'})
            ]

    elif (input_year and selected_statistics=='Yearly Statistics'):
        #Filter the data based on input year
        yearly_data = data[data['Year'] == input_year]

        #Create and display graphs for yearly statistics
        yearly_sales = data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(figure=px.line(yearly_sales, x='Year', y='Automobile_Sales', title='Yearly Average Automobile Sales'))


        tot_month_sales = yearly_data.groupby('Month')['Automobile_Sales'].sum().reset_index()
        Y_chart2 = dcc.Graph(figure=px.line(tot_month_sales, x='Month', y='Automobile_Sales', title='Total Monthly Automobile Sales in {}'.format(input_year)))

        month_sales = yearly_data.groupby(['Month', 'Vehicle_Type'], as_index=False)['Automobile_Sales'].mean()
        Y_chart3 = dcc.Graph(figure=px.bar(month_sales, x='Month', y='Automobile_Sales', color='Vehicle_Type', title='Average Vehicles Sold by Vehicle Type in {}'.format(input_year)))

        exp_data = yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        Y_chart4 = dcc.Graph(figure=px.pie(exp_data, values='Advertising_Expenditure', names='Vehicle_Type', title='Total Advertisement Expenditure for Each Vehicle Type in {}'.format(input_year)))

        return [
            html.Div(className='chart-item', children=[html.Div(children=Y_chart1),html.Div(children=Y_chart2)], style={'display': 'flex'}),
            html.Div(className='chart-item', children=[html.Div(children=Y_chart3),html.Div(children=Y_chart4)], style={'display': 'flex'})
            ]

    else:
        return None

if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>